In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
#from train import create_dir, load_dataset

In [2]:
global image_h
global image_w
global num_landmarks

In [3]:
""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

""" Directory for storing files """
# create_dir("results")

""" Hyperparameters """
image_h = 512
image_w = 512
num_landmarks = 106

""" Paths """
path = "./LaPa"
model_path = os.path.join("files_test", "model.h5")

In [4]:
def plot_landmarks(image, landmarks):
    h, w, _ = image.shape
    radius = int(h * 0.005)

    for i in range(0, len(landmarks), 2):
        x = int(landmarks[i] * w)
        y = int(landmarks[i+1] * h)

        image = cv2.circle(image, (x, y), radius, (255, 0, 0), -1)

    return image

In [5]:
""" Load the model """
model = tf.keras.models.load_model(model_path)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 256, 256, 16  432         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 256, 256, 16  64          ['Conv1[0][0]']                  
                                )                                                             

                                                                                                  
 block_3_expand (Conv2D)        (None, 128, 128, 96  1536        ['block_2_add[0][0]']            
                                )                                                                 
                                                                                                  
 block_3_expand_BN (BatchNormal  (None, 128, 128, 96  384        ['block_3_expand[0][0]']         
 ization)                       )                                                                 
                                                                                                  
 block_3_expand_relu (ReLU)     (None, 128, 128, 96  0           ['block_3_expand_BN[0][0]']      
                                )                                                                 
                                                                                                  
 block_3_p

                                                                                                  
 block_6_depthwise_BN (BatchNor  (None, 32, 32, 96)  384         ['block_6_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_6_depthwise_relu (ReLU)  (None, 32, 32, 96)   0           ['block_6_depthwise_BN[0][0]']   
                                                                                                  
 block_6_project (Conv2D)       (None, 32, 32, 32)   3072        ['block_6_depthwise_relu[0][0]'] 
                                                                                                  
 block_6_project_BN (BatchNorma  (None, 32, 32, 32)  128         ['block_6_project[0][0]']        
 lization)                                                                                        
          

 lization)                                                                                        
                                                                                                  
 block_10_expand_relu (ReLU)    (None, 32, 32, 192)  0           ['block_10_expand_BN[0][0]']     
                                                                                                  
 block_10_depthwise (DepthwiseC  (None, 32, 32, 192)  1728       ['block_10_expand_relu[0][0]']   
 onv2D)                                                                                           
                                                                                                  
 block_10_depthwise_BN (BatchNo  (None, 32, 32, 192)  768        ['block_10_depthwise[0][0]']     
 rmalization)                                                                                     
                                                                                                  
 block_10_

 alization)                                                                                       
                                                                                                  
 block_14_expand (Conv2D)       (None, 16, 16, 480)  38400       ['block_13_project_BN[0][0]']    
                                                                                                  
 block_14_expand_BN (BatchNorma  (None, 16, 16, 480)  1920       ['block_14_expand[0][0]']        
 lization)                                                                                        
                                                                                                  
 block_14_expand_relu (ReLU)    (None, 16, 16, 480)  0           ['block_14_expand_BN[0][0]']     
                                                                                                  
 block_14_depthwise (DepthwiseC  (None, 16, 16, 480)  4320       ['block_14_expand_relu[0][0]']   
 onv2D)   

 dense (Dense)                  (None, 212)          271572      ['dropout[0][0]']                
                                                                                                  
Total params: 977,796
Trainable params: 959,252
Non-trainable params: 18,544
__________________________________________________________________________________________________


In [6]:
x = './LaPa/test/onebin.jpg'
x

'./LaPa/test/onebin.jpg'

In [7]:
""" Reading the image """
image = cv2.imread(x, cv2.IMREAD_COLOR)
image_x = image

image = cv2.resize(image, (image_w, image_h))

image = image/255.0 ## (512, 512, 3)
image = np.expand_dims(image, axis=0) ## (1, 512, 512, 3)
image = image.astype(np.float32)
image

array([[[[0.05882353, 0.09019608, 0.11764706],
         [0.05882353, 0.09019608, 0.11764706],
         [0.05882353, 0.09019608, 0.11764706],
         ...,
         [0.39215687, 0.5647059 , 0.6862745 ],
         [0.39607844, 0.5686275 , 0.6901961 ],
         [0.39607844, 0.5686275 , 0.6901961 ]],

        [[0.05490196, 0.08627451, 0.11372549],
         [0.05490196, 0.08627451, 0.11372549],
         [0.05490196, 0.08627451, 0.11372549],
         ...,
         [0.39607844, 0.5686275 , 0.6901961 ],
         [0.39607844, 0.5686275 , 0.6901961 ],
         [0.39607844, 0.5686275 , 0.6901961 ]],

        [[0.05490196, 0.08627451, 0.11372549],
         [0.05490196, 0.08627451, 0.11372549],
         [0.05490196, 0.08627451, 0.11372549],
         ...,
         [0.39607844, 0.5686275 , 0.6901961 ],
         [0.39607844, 0.5686275 , 0.6901961 ],
         [0.39607844, 0.5686275 , 0.6901961 ]],

        ...,

        [[0.1764706 , 0.30588236, 0.43529412],
         [0.18039216, 0.30980393, 0.4392157 ]

In [8]:
""" Prediction """
pred = model.predict(image, verbose=0)[0]
pred = pred.astype(np.float32)
pred

array([0.2895909 , 0.50438195, 0.28336436, 0.5376349 , 0.2768091 ,
       0.5713519 , 0.27406263, 0.60347885, 0.27409396, 0.6378427 ,
       0.2742042 , 0.6711133 , 0.2793125 , 0.7039632 , 0.28602117,
       0.73636544, 0.29729915, 0.76625466, 0.31216574, 0.79434913,
       0.3316584 , 0.82070917, 0.35296452, 0.8446814 , 0.3783122 ,
       0.8654277 , 0.40740502, 0.88397694, 0.4401189 , 0.898677  ,
       0.47662458, 0.90663254, 0.5130979 , 0.9074417 , 0.5408462 ,
       0.8989542 , 0.56058663, 0.8838022 , 0.57559174, 0.8632438 ,
       0.58772147, 0.8420293 , 0.60296905, 0.8201557 , 0.6158554 ,
       0.7980587 , 0.6277007 , 0.7743253 , 0.63827896, 0.7499292 ,
       0.6487263 , 0.7252858 , 0.6583789 , 0.70089835, 0.6677271 ,
       0.676057  , 0.6743128 , 0.6499064 , 0.67749566, 0.62346476,
       0.67929924, 0.5979654 , 0.68089855, 0.57274324, 0.68269485,
       0.54741114, 0.3791342 , 0.49211538, 0.4133449 , 0.4690632 ,
       0.4544149 , 0.4661887 , 0.49587575, 0.47952858, 0.52953

In [9]:
""" Saving the results """
#gt_landmarks = plot_landmarks(image_x.copy(), landmarks)
result_image = plot_landmarks(image_x.copy(), pred)
# line = np.ones((image_x.shape[0], 10, 3)) * 255

In [12]:
print(type(result_image))

<class 'numpy.ndarray'>


In [13]:
result_image

array([[[ 15,  23,  30],
        [ 15,  23,  30],
        [ 15,  23,  30],
        ...,
        [100, 144, 175],
        [101, 145, 176],
        [101, 145, 176]],

       [[ 14,  22,  29],
        [ 14,  22,  29],
        [ 14,  22,  29],
        ...,
        [101, 145, 176],
        [101, 145, 176],
        [101, 145, 176]],

       [[ 14,  22,  29],
        [ 14,  22,  29],
        [ 14,  22,  29],
        ...,
        [101, 145, 176],
        [101, 145, 176],
        [101, 145, 176]],

       ...,

       [[ 45,  78, 111],
        [ 46,  79, 112],
        [ 45,  80, 113],
        ...,
        [125, 147, 165],
        [133, 155, 173],
        [130, 152, 170]],

       [[ 44,  77, 110],
        [ 46,  79, 112],
        [ 47,  80, 113],
        ...,
        [128, 150, 168],
        [125, 147, 165],
        [121, 143, 161]],

       [[ 44,  77, 110],
        [ 46,  79, 112],
        [ 47,  80, 113],
        ...,
        [128, 150, 168],
        [122, 144, 162],
        [119, 141, 159]]

In [15]:
# cat_image = np.concatenate([gt_landmarks, line, pred_landmarks], axis=1)
name = "onebin"
cv2.imwrite(f"./result2/{name}.png", result_image)

True